In [1]:
pip install flask-sqlalchemy

  Obtaining dependency information for flask-sqlalchemy from https://files.pythonhosted.org/packages/1d/6a/89963a5c6ecf166e8be29e0d1bf6806051ee8fe6c82e232842e3aeac9204/flask_sqlalchemy-3.1.1-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask , redirect , url_for , render_template ,flash , request , session
from datetime import timedelta
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.secret_key = "hello"
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.sqlite3'
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
app.permanent_session_lifetime = timedelta(minutes = 5)

db = SQLAlchemy(app)

class users(db.Model) :
    _id = db.column("id",db.Integer , primary_key = True)
    name = db.column( db.String(100))
    email = db.column(db.String(100))
    
    def __init__(self , name , email) :
        self.name = name
        self.email = email

@app.route("/")
def home() :
    return render_template("index.html")

@app.route("/login" , methods = ["GET" , "POST"])
def login() :
    if request.method == "POST" :
        session.permanent = True
        user = request.form["nm"]
        session["user"] = user
        
        flash("Login Sucessful" , "info")
        return redirect(url_for("user"))
    
    else :
        if "user" in  session :
            flash("Already Logged In!!" , "info")
            return redirect(url_for("user"))
        else :
            return render_template("login.html")
    
@app.route("/user" , methods = ["POST" , "GET"])
def user() :
    email = None
    if "user" in session :
        user = session["user"]
        
        if request.method == "POST" :
            email = request.form["email"]
            session["email"] = email
            flash("Email was saved!")
            
        else :
            if "email" in session :
                email = session["email"]
        return render_template("user.html" , email = email )
    else :
        flash("You are not Logged In !" , "info")
        return render_template("login.html")
    

@app.route("/logout")
def logout() :
    if "user" in session :
        user = session["user"]
        flash(f"You have been loggged out , {user}" , "info")
    session.pop("user" , None)
    session.pop("email" , None)
    return render_template("login.html")

if __name__ == "__main__" :
    db.create_all()
    app.run(debug = True , use_reloader = False , port = 7000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:7000
Press CTRL+C to quit
127.0.0.1 - - [21/May/2024 16:00:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2024 16:00:37] "GET /login HTTP/1.1" 302 -
127.0.0.1 - - [21/May/2024 16:00:37] "GET /user HTTP/1.1" 200 -
127.0.0.1 - - [21/May/2024 16:00:39] "POST /user HTTP/1.1" 200 -
